In [ ]:
import pandas as pd
import requests
import numpy as np
import time
import utils
import openpyxl
import regex as re
import tokens

In [ ]:
box_path = "../../Library/CloudStorage/Box-Box/EEOC data/"

Create a dataset linking states, counties, and FIPS codes

In [ ]:
#the DP05_0058E part of the query is just a placeholder--we're doing this so we have a county name-county fip crosswalk
q = f"https://api.census.gov/data/2014/acs/acs5/profile?get=GEO_ID,NAME,DP05_0058E&for=county:*&in=state:*&key={utils.CENSUS_KEY}"
pop_r = requests.get(q)
pop_resp = pop_r.json()
headers = pop_resp.pop(0)
county_fips_data = pd.DataFrame(pop_resp, columns=headers)
county_fips_data = county_fips_data.rename(columns = {"GEO_ID":"geo_id",
                                  "NAME": "county_state",
                                  "DP05_0058E": "total_pop",
                                  "state": "state_fips",
                                  "county": "county_fips"})
county_fips_data["CountyFIPS"] = county_fips_data["state_fips"]+county_fips_data["county_fips"]

county_fips_data = county_fips_data[["geo_id","CountyFIPS", "county_state", "state_fips", "county_fips"]]

def get_county_name(county_state):
    cty_str = county_state.split(",")[0]
    cty_str = cty_str.replace(" County", "")
    return cty_str
def get_state_name(county_state):
    state_str = county_state.split(",")[1]
    state_str = state_str.strip()
    return state_str

county_fips_data["CountyName"] = county_fips_data["county_state"].apply(get_county_name)
county_fips_data["StateName"] = county_fips_data["county_state"].apply(get_state_name)
county_fips_data["StateAbbr"] = county_fips_data.StateName.map(utils.us_state_abbrev)
county_fips_data["CountyName_low"] = county_fips_data["CountyName"].str.lower()


Read in the dataset with latitudes and longitudes for US zip codes

In [ ]:
def fix_zip(old_zip):
    if pd.isnull(old_zip):
        return
    if type(old_zip) != str:
        old_zip = str(int(old_zip))
    if any(s.isalpha() for s in old_zip):
        return
    if "-" in old_zip:
        old_zip = old_zip[0:re.search("[-]", old_zip).span()[0]]
    
    return old_zip.zfill(5)

#https://www.unitedstateszipcodes.org/zip-code-database/
all_zips = pd.read_csv(box_path+"zip_code_database.csv")

all_zips["fixed_zip"] = all_zips["zip"].apply(fix_zip)
zips = all_zips[all_zips["state"].isin(list(utils.us_state_abbrev.values()))]
zips["county"] = zips["county"].str.replace(" County", "")
zips["county_low"] = zips["county"].str.lower()

Read in the dataset with unique zipcodes

In [ ]:
unique_places = pd.read_excel(box_path+"places_to_geocode.xlsx", 
                              engine = "openpyxl")

## Step 0: Clean the locations to prepare them for geocoding

This involves salvaging any zipcodes that were incorrectly entered as the city, removing cases that are not within the 50 U.S. states or Washington, D.C., and fixing city names that were originally misspelled and identified via manual review. The city names that need to be fixed manually will become apparent in **Step 3**. You can add the manual fix cities to a reference spreadsheet (I've named mine `city_recode.xlsx`). The city and state combinations that need to be fixed manually are saved in a separate spreadsheet (`manual_fix_citystates.xlsx`).

In [ ]:
def salvage_zipcodes(city_state, old_zip):
    if pd.isnull(old_zip) and pd.isnull(city_state)==False:
        if re.search("\d{5}[,] [A-Z][a-z]", city_state):
            if not re.search("\d{6}[,] [A-Z][a-z]", city_state):
                return re.search("\d{5}", city_state).group()
            else:
                return
        else:
            return
    elif pd.isnull(old_zip)==False:
        if re.search("\d{5}", old_zip):
            if not re.search("\d{6}", old_zip):
                return re.search("\d{5}", old_zip).group()
            else:
                return
    return old_zip

In [ ]:
unique_places["respondent_zip_clean"] = unique_places.apply(lambda x: salvage_zipcodes(x["city_state"],
                                                                                   x["respondent_zip_nn"]),
                                                        axis = 1)

In [ ]:
def score_state(state):
    if pd.isnull(state)==False:
        if state == "Puerto Rico" or "FPO" in state or "APO" in state or "Guam" in state or "US Virgin Islands" in state or "Marianna" in state:
            return -1
        else:
            return 1
    else:
        return 0
def score_city(city):
    if pd.isnull(city) == False:
        if re.search("[(]\d+\d+", city):
            return 0
        elif re.search("\d+", city[0:1]):
            return 0
        elif len(city) == 1:
            return 0
        else:
            return 1
    else:
        return 0
def score_county(county):
    if pd.isnull(county)==False:
        return 1
    else:
        return 0
def score_zip(zipcode):
    if pd.isnull(zipcode)==False:
        return 1
    return 0

def get_loc_score(city_score, state_score, zip_score, county_score):
    if state_score ==1:
        if city_score == 1 or county_score == 1 or zip_score == 1:
            return 1
        else:
            return 0
    else:
        if state_score <0:
            return 0
        if zip_score ==1:
            return 1
    return 0

"Score" the locations based on whether it has enough information to get a lat/long

In [ ]:
unique_places["city_score"] = unique_places["respondent_city_nn"].apply(score_city)
unique_places["zip_score"] = unique_places["respondent_zip_clean"].apply(score_zip)
unique_places["county_score"] = unique_places["respondent_county_nn"].apply(score_county)
unique_places["state_score"] = unique_places["respondent_state_nn"].apply(score_state)

In [ ]:
unique_places["loc_score"] = unique_places.apply(lambda x: get_loc_score(x["city_score"],
                                                                  x["state_score"],
                                                                  x["zip_score"],
                                                                  x["county_score"]),
                                          axis = 1)

In [ ]:
state_abbrevs = dict(zip(list(utils.us_state_abbrev.values()),
                         list(utils.us_state_abbrev.keys())))

Remove any locations outside the US or in an APO/FPO 

In [ ]:
unique_places.loc_score.value_counts()

In [ ]:
unique_places = unique_places[unique_places.loc_score > 0]

In [ ]:
unique_places["respondent_state_abbrev_nn"] = unique_places["respondent_state_nn"].map(utils.us_state_abbrev)
unique_places["state_in_us"] = unique_places["respondent_state_abbrev_nn"].isnull()==False

In [ ]:
unique_places.loc[pd.isnull(unique_places["respondent_county_nn"])==False, "state_in_us"] = True


Remove any locations that aren't within the 50 US states or Washington D.C.

In [ ]:
remove = ["Guam", "US Virgin Islands", "Palau", "Northern Marianna Islands", "Puerto Rico",
         "American Samoa"]
unique_places.loc[unique_places["respondent_state_nn"].astype(str).isin(remove), "state_in_us"] = False

In [ ]:
unique_places["state_in_us"].value_counts()

Remove the non-US states from the "state" column

In [ ]:
unique_places["respondent_state_fixed"] = unique_places["respondent_state_nn"]

In [ ]:
unique_places.loc[unique_places["respondent_state_nn"].astype(str)=="District Of Columbia", "state_in_us"] = True
unique_places.loc[unique_places["respondent_state_nn"].astype(str)=="District of Columbia", "state_in_us"] = True

In [ ]:
unique_places["state_in_us"].value_counts()

In [ ]:
unique_places.loc[unique_places["state_in_us"]==False, "respondent_state_fixed"] = None

In [ ]:
unique_places.loc[unique_places["respondent_state_fixed"].astype(str).isin(remove), "respondent_state_fixed"] = None

In [ ]:
unique_places["zip_state"] = unique_places["respondent_zip_clean"].astype(str) +", "+ unique_places["respondent_state_fixed"].astype(str)

Read in spreadsheets to fix spelling errors on cities and states

In [ ]:
city_fix_df = pd.read_excel(box_path+"city_recode.xlsx", engine = "openpyxl")
recode_cities = dict(zip(city_fix_df["original_city"].to_list(), 
                        city_fix_df["new_city"].to_list()))
cities_and_states = pd.read_excel(box_path + "manual_fix_citystates.xlsx", engine = "openpyxl")
cities_and_states["city_state"] = cities_and_states["city_state"].str.lower()
cities_and_states["city_state_clean"] = cities_and_states["city_state_clean"].str.lower()

recode_city_states = dict(zip(cities_and_states["city_state"].to_list(),
                             cities_and_states["city_state_clean"].to_list()))

Run the function to handle incorrectly-spelled locations

In [ ]:

def fix_fip(x):
    if pd.isnull(x):
        return ""
    else:
        x = str(int(float(x)))
        return x.zfill(5)

def fix_city(x):
    replace_strings = ["St[.] ", "St ", "Ft[.] ", "Ft ", "Mt[.] ", "Mt "]
    new_strings = ["Saint ", "Saint ", "Fort ","Fort ", "Mount ", "Mount "]
    replace_list = "|".join(replace_strings)
    replace_list = "("+replace_list+")"
    if "Hieghts" in x:
        x = x.replace("Hieghts", "Heights")
    if pd.isnull(x):
        return
    else:
        x = x.rstrip()
        x = x.lstrip()
        x = re.sub("\s{2,6}", " ", x)
        if re.search(replace_list, x):
            for i in range(0, len(new_strings)):
                if re.search(replace_strings[i], x):
                    x = re.sub(replace_strings[i], new_strings[i], x)
                    #return x
        if x in recode_cities.keys():
            x = recode_cities[x]
        x = re.sub(",", "", x)
        return x
    
def fix_city_state(x):
    if pd.isnull(x):
        return
    x = re.sub("\s{2,6}", " ", x)
    if x in recode_city_states.keys():
        x = recode_city_states[x]
    return x
        
def fix_city_and_state(city, state, return_type = "city_state"):
    if pd.isnull(city) or pd.isnull(state):
        return None, None
    city = fix_city(city)
    if city in recode_cities.keys():
        city = recode_cities[city]
    city_state = city + ", " + state
    city_state = city_state.lower()
    if city_state in recode_city_states.keys():
        city_state = recode_city_states[city_state]
        city = city_state.split(",")[0]
        city = city.lstrip()
        city = city.rstrip()
    if ("district of columbia" in state.lower() or 
        "district of columbia" in city.lower()):
        city = "Washington"
        city_state = "washington, district of columbia"
        
    return city, city_state
    
    


In [ ]:
unique_places["city"], unique_places["city_state_clean"] = zip(*unique_places.apply(lambda x: fix_city_and_state(x["respondent_city_nn"],
                                                                             x["respondent_state_fixed"]),axis=1))

In [ ]:
unique_places = unique_places[unique_places["state_in_us"]==True]

## Step 1: Merge the places dataset with the county FIP dataset based on respondent county

In [ ]:
unique_places["respondent_county_nn_low"] = unique_places["respondent_county_nn"].str.lower()

In [ ]:

unique_places_merge = pd.merge(unique_places, 
                         county_fips_data[["StateName", "CountyFIPS","CountyName_low"]], 
                         left_on = ["respondent_county_nn_low", "respondent_state_fixed"],
                         right_on = ["CountyName_low", "StateName"], 
                         how = "left")


In [ ]:
unique_places_merge = unique_places_merge.drop(columns = ["StateName", "CountyName_low"])


In [ ]:
unique_places_merge = unique_places_merge.rename(columns = {"CountyFIPS":"state_county_fips"})

## Step 2: Merge the places dataset with the lat/long dataset based on respondent zip code

Here, exclude any places with a missing zipcode

In [ ]:
#create an identifier for the places
unique_places_merge = unique_places_merge.reset_index(drop = True)
unique_places_merge["place_id"] = unique_places_merge.index.values

In [ ]:
len(unique_places_merge)

In [ ]:
unique_places_merge.respondent_state_abbrev_nn

In [ ]:
zip_merge = pd.merge(unique_places_merge, zips, how = "left", 
                      left_on = ["respondent_zip_clean","respondent_state_abbrev_nn"], 
                     right_on = ["fixed_zip", "state"])
zip_merge['latitude'].isnull().sum()

Separate out the locations that successfully merged on county name and zip code from those that did not

In [ ]:
zip_merge["success"] = pd.isnull(zip_merge["latitude"])==False
zip_merge.loc[pd.isnull(zip_merge["state_county_fips"])==False, "success"] = True

In [ ]:
zip_merge["county"] = zip_merge["county"].str.replace(" County", "")

In [ ]:
zip_merge["county_low"] = zip_merge["county"].str.lower()

In [ ]:
print(zip_merge.state_county_fips.isnull().sum())
zip_merge = pd.merge(zip_merge, county_fips_data[["CountyName_low", "StateAbbr", "CountyFIPS"]],
                    left_on = ["county_low", "state"],
                    right_on = ["CountyName_low", "StateAbbr"],
                    how="left")
print(zip_merge.CountyFIPS.isnull().sum())

In [ ]:
zip_merge.loc[pd.isnull(zip_merge.state_county_fips), "state_county_fips"] = zip_merge["CountyFIPS"]

In [ ]:
zip_merge["state_county_fips"].isnull().sum()

In [ ]:
zip_merge= zip_merge.drop(columns = ["CountyName_low", "StateAbbr","CountyFIPS"])

In [ ]:
zips["state_full"] = zips["state"].map(state_abbrevs)

In [ ]:
zips_crossref = zips.copy()
zips_crossref["city_state_clean"] = zips_crossref["primary_city"]+", "+zips_crossref["state_full"]
zips_crossref["city_state_lower"] = zips_crossref['city_state_clean'].str.lower()

## Step 3: Merge places dataset with zipcode dataset and county FIP dataset based on city and state

For places that failed to merge on zip codes alone, try to match them to city strings. This section first estimates the correct county name for each failed location based on the city and state name, and then locates the county FIP code using the estimated county name.

In [ ]:
fail_df = pd.DataFrame() #track the places that still fail based on the city + state search
tracker = 0

In [ ]:
zip_merge["city"] = zip_merge["city"].str.lower()

In [ ]:
zip_merge = zip_merge.reset_index(drop = True)

In [ ]:
def get_all_acceptable(primary, others):
    if pd.isnull(others):
        return primary
    else:
        return primary + ', ' + others

In [ ]:
zips_crossref["all_acceptable"] = zips_crossref.apply(lambda x: get_all_acceptable(x["primary_city"],
                                                                                  x["acceptable_cities"]),
                                                     axis = 1)

In [ ]:
check_locs = ["north ", "east ", "west ", "south ",
                         "n. ", "e. ", "w. ", "s. "]
for i in zip_merge.index:
    if pd.isnull(zip_merge.loc[i, "state_county_fips"])==False:
        continue
    #try to find a lat/long match for the city
    if pd.isnull(zip_merge.loc[i, "state_county_fips"]) and (pd.isnull(zip_merge.loc[i, "city_state_clean"])==False):
        look = zip_merge.loc[i, "city_state_clean"].lower()
        look = look.lstrip()
        look = look.rstrip()
        if look in zips_crossref["city_state_lower"].to_list():
            tmp = zips_crossref.loc[zips_crossref["city_state_lower"].str.contains(look),:]
            tmp = tmp.reset_index(drop= True)
            if len(tmp) > 0:
                #print("FOUND: ", zip_merge.loc[i, "city_state_clean"])
                for c in zips_crossref.columns:
                    if c != "city_state_lower" and c != "city_state_clean":
                        zip_merge.loc[i, c] = tmp.loc[0, c]
            else:
                tmp = zips_crossref.loc[zips_crossref["city_state_lower"]==look,:]
                tmp = tmp.reset_index(drop= True)
                if len(tmp) > 0:
                    #print("FOUND: ", zip_merge.loc[i, "city_state_clean"])
                    for c in zips_crossref.columns:
                        if c != "city_state_lower" and c != "city_state_clean":
                            zip_merge.loc[i, c] = tmp.loc[0, c]
        elif any(zip_merge.loc[i, "city"].startswith(l) for l in check_locs):
            for l in check_locs:
                if zip_merge.loc[i, "city"].startswith(l):
                    
                    look = zip_merge.loc[i, "city_state_clean"].replace(l, "")
                    look = look.lower()
                    if look in zips_crossref["city_state_lower"].to_list():
                        tmp = zips_crossref.loc[zips_crossref["city_state_lower"].str.contains(look),:]
                        tmp = tmp.reset_index(drop= True)
                        if len(tmp) > 0:
                            #print("FOUND: ", zip_merge.loc[i, "city_state_clean"])
                            for c in zips_crossref.columns:
                                if c != "city_state_lower" and c != "city_state_clean":
                                    zip_merge.loc[i, c] = tmp.loc[0, c]
                            break
        else:
            look = zip_merge.loc[i, "city"].lower().lstrip().rstrip()
            tmp = zips_crossref.loc[zips_crossref["state"] ==zip_merge.loc[i, "respondent_state_abbrev_nn"],:]
            tmp["acceptable_city"] = tmp["all_acceptable"].apply(lambda x: look in x.lower() if type(x)==str else False)
            tmp = tmp.loc[tmp["acceptable_city"]==True,:]
            tmp = tmp.reset_index(drop = True)
            if len(tmp) > 0:
                #print("FOUND: ", zip_merge.loc[i, "city_state_clean"])
                for c in zips_crossref.columns:
                    if c != "city_state_lower" and c != "city_state_clean":
                        zip_merge.loc[i, c] = tmp.loc[0, c]
                        
            if pd.isnull(zip_merge.loc[i, "latitude"]):
                for c in zip_merge.columns:
                    fail_df.loc[tracker, c] = zip_merge.loc[i, c]
                tracker += 1
                
                print("FAILED: ", zip_merge.loc[i, "city_state"], 
                      "CLEAN NAME: ", zip_merge.loc[i, "city_state_clean"],
                      " COUNTY: ", zip_merge.loc[i, "respondent_county_nn"])
                
                
                

Now that we've found any places that might have a misspecified state, we'll try merging again based on zip code and city name or county name (whichever is available), this time ignoring the state. This is not as error-prone now that any positive matches based on the searches we've done so far are probably due to a misspecified city name or state name. 

We'll print out the corresponding primary cities and alternative cities for the zip code to see if the places names might match despite having a different state (indicating that the state was misspecified in error since both the zip code and the place name correspond). 

In [ ]:
zip_merge_zips = zip_merge.copy()

In [ ]:
zips_crossref["acceptable_w_county"] = zips_crossref["all_acceptable"] + ", " + zips_crossref["county"]

In [ ]:
#look for cases where the state might have been incorrectly specified
for i in zip_merge_zips.index:
    if pd.isnull(zip_merge_zips.loc[i, "state_county_fips"])==False:
        continue
    if pd.isnull(zip_merge_zips.loc[i, "latitude"])==False:
        continue
    if pd.isnull(zip_merge_zips.loc[i, "respondent_zip_clean"])==False:
        tmp = zips_crossref.loc[zips_crossref["fixed_zip"]==zip_merge_zips.loc[i, "respondent_zip_clean"],:]
        tmp = tmp.reset_index(drop = True)
        if pd.isnull(zip_merge_zips.loc[i, "city"])==False:
            look = zip_merge_zips.loc[i, "city"].lower()
        elif pd.isnull(zip_merge_zips.loc[i, "respondent_county_nn"])==False:
            look = zip_merge_zips.loc[i, "respondent_county_nn"].lower()
        elif pd.isnull(zip_merge_zips.loc[i, "respondent_city_nn"])==False:
            look = zip_merge_zips.loc[i, "respondent_city_nn"].lower()
        else:
            print("nothing to search for: ", zip_merge_zips.loc[i, "respondent_county_nn"],
                 " ", zip_merge_zips.loc[i, "city"])
            continue
        if len(tmp) > 0:
            tmp["acceptable_city"] = tmp["acceptable_w_county"].apply(lambda x: 
                                                                      look in x.lower() if type(x)==str else False)
            
            if tmp.loc[0, "acceptable_city"]==True:
                print(" -------")
                print("FOUND: ", zip_merge_zips.loc[i, "city_state_clean"])
                print("Acceptable cities: ", tmp.loc[0, "acceptable_cities"])
                print("Primary cities: ", tmp.loc[0, "primary_city"])
                print("County: ", tmp.loc[0, "county"])
                print("State abb.: ", tmp.loc[0, "state"])
                print(" -------")
                print("  \n")
                for c in zips_crossref.columns:
                    if c != "city_state_lower" and c != "city_state_clean":
                        zip_merge_zips.loc[i, c] = tmp.loc[0, c]
        

## Step 4: Use the fixed county name information from the city-based search to get the FIP code

In [ ]:
print(zip_merge_zips.state_county_fips.isnull().sum())
zip_merge_zips["county"] = zip_merge_zips["county"].str.replace(" County", "")
zip_merge_zips["county_lower"] = zip_merge_zips["county"].str.lower()
zip_merge_1 = pd.merge(zip_merge_zips, 
                       county_fips_data[["CountyName_low", "StateAbbr", "CountyFIPS"]],
                    left_on = ["county_lower", "state"],
                    right_on = ["CountyName_low", "StateAbbr"],
                    how="left")
print(zip_merge_1.CountyFIPS.isnull().sum())

In [ ]:
zip_merge_1.loc[pd.isnull(zip_merge_1.state_county_fips), "state_county_fips"] = zip_merge_1["CountyFIPS"]

In [ ]:
zip_merge_1["state_county_fips"].isnull().sum()

In [ ]:
zip_merge_1 = zip_merge_1.drop(columns = ["CountyName_low", "StateAbbr", "CountyFIPS"])

Now that most locations have been matched pretty exhaustively, we'll use just the zip code of the locations (without the state name) to fill in any info that's still missing but probably has misspecified state name. 

In [ ]:
zip_merge_1 = pd.merge(zip_merge_1, 
                     zips[["county", "state", "fixed_zip","longitude", "latitude"]], 
                     how = "left", 
                      left_on = ["respondent_zip_nn"], 
                     right_on = ["fixed_zip"],
                                suffixes = ("", "_nostate"))
fix_cols = ["county", "state", "fixed_zip","longitude", "latitude"]
for f in fix_cols:
    print(f)
    print(zip_merge_1[f].isnull().sum())
    zip_merge_1.loc[pd.isnull(zip_merge_1.latitude), f] = zip_merge_1[f+"_nostate"]
    print(zip_merge_1[f].isnull().sum())
zip_merge_1 = zip_merge_1.drop(columns = [f+"_nostate" for f in fix_cols])

In [ ]:
print(zip_merge_1.state_county_fips.isnull().sum())
zip_merge_1["county"] = zip_merge_1["county"].str.replace(" County", "")
zip_merge_1["county_lower"] = zip_merge_1["county"].str.lower()
zip_merge_1 = pd.merge(zip_merge_1, 
                       county_fips_data[["CountyName_low", "StateAbbr", "CountyFIPS"]],
                    left_on = ["county_lower", "state"],
                    right_on = ["CountyName_low", "StateAbbr"],
                    how="left")
print(zip_merge_1.CountyFIPS.isnull().sum())

In [ ]:
zip_merge_1.loc[pd.isnull(zip_merge_1.state_county_fips), "state_county_fips"] = zip_merge_1["CountyFIPS"]

In [ ]:
print(zip_merge_1.state_county_fips.isnull().sum())

## Step 5: Use place lat/long to get county FIP code

Get the county FIPS codes for each place using the FCC API if the latitude/longitude of the place is available but failed to match with a county in the county FIP crosswalk dataset. You'll need a Census API key to query the FCC API (saved in a `tokens.py` file)

In [ ]:
def fcc_api_wrapper(lat, long):
    fcc_req = (f'https://geo.fcc.gov/api/census/block/find?latitude={lat}&longitude={long}&showall'+
                                      f'=false&format=json&key={tokens.CENSUS_KEY}')
    resp = requests.get(fcc_req)
    try:
        result = resp.json()
    except:
        print('error in fip retrieval: ', resp)
        print(type(resp))
        return  np.nan, np.nan
    try:
        state_county_fips = result['County']['FIPS']
        county = result['County']['name']

    except:
        print('error, could not retrieve fips for: ', lat, long)
        return None, None
    return state_county_fips, county

In [ ]:
import time

In [ ]:
zip_merge_1["lat_long"] = zip_merge_1["latitude"].astype(str)+"_"+zip_merge_1["longitude"].astype(str)

In [ ]:
len(zip_merge_1)

In [ ]:
zip_merge_1["state_county_fips"].isnull().sum()

In [ ]:
start = time.time()
for i in zip_merge_1.index:
    if pd.isnull(zip_merge_1.loc[i, "latitude"]):
        continue
    elif pd.isnull(zip_merge_1.loc[i, "state_county_fips"])==False:
        continue
    else:
        fips, county_name = fcc_api_wrapper(zip_merge_1.loc[i, "latitude"], zip_merge_1.loc[i, "longitude"])
        zip_merge_1.loc[i, "state_county_fips"] = fips
        zip_merge_1.loc[i, "fcc_county_name"] = county_name
    if i % 500 == 0:
        print("time: ", time.time()-start)
        print(i)

In [ ]:
zip_merge_1["state_county_fips"].isnull().sum()

In [ ]:
zip_merge_1["latitude"].isnull().sum()

Save the places for which a FIP code could not be identified

In [ ]:
fail_final = zip_merge_1[pd.isnull(zip_merge_1["state_county_fips"])]

In [ ]:
writer = pd.ExcelWriter(box_path+"failed_geocode.xlsx", engine = 'xlsxwriter')
fail_final.to_excel(writer, encoding = "utf-8-sig", index = False)
writer.close()

## Step 6: Manually fix failed locations

To get the county name of the failed locations (saved as `failed_geocode_new_method_v2.xlsx`), we searched for the place name to get the associated county and then referenced the county FIP crosswalk spreadsheet we've been using thus far. We saved the version of the dataset with the retrieved county names as `manual_countynames.xlsx` to avoid overwriting the file.

In [ ]:
manual_counties = pd.read_excel(box_path+"manual_countynames.xlsx", engine = "openpyxl")

In [ ]:
manual_counties["respondent_county_low"] = manual_counties.respondent_county_nn.str.lower()
manual_counties = pd.merge(manual_counties,
                          county_fips_data[["CountyName_low", "StateName","CountyFIPS"]],
                          left_on = ["respondent_county_low", "respondent_state_nn"],
                          right_on = ["CountyName_low", "StateName"],
                          how = "left")

In [ ]:
manual_counties = manual_counties.dropna(subset = ["CountyFIPS"])
manual_counties = manual_counties[["charge_unique_numbers", "CountyFIPS"]]
manual_counties.columns = ["charge_unique_numbers", "state_county_fips"]

In [ ]:
manual_counties

In [ ]:
manual_dict = dict(zip(manual_counties["charge_unique_numbers"].to_list(),
                      manual_counties["state_county_fips"].to_list()))

In [ ]:
zip_merge_1["CountyFips_man"] = zip_merge_1["charge_unique_numbers"].map(manual_dict)

In [ ]:
zip_merge_1.state_county_fips.isnull().sum()

In [ ]:
zip_merge_1.loc[pd.isnull(zip_merge_1.state_county_fips), "state_county_fips"] = zip_merge_1["CountyFips_man"]
zip_merge_1.state_county_fips.isnull().sum()

## Final step: Create an easy-to-merge dataset to assign county FIP codes to cases

This step "explodes" the list of unique charge numbers identifying each case so that the geocoded locations can be perfectly matched with the charges dataset. It will result in a much longer dataset than the original geocoded places dataset because it essentially duplicates any locations with more than one charge within it for easier merging.

In [ ]:
zip_merge_1["charge_nums"] = zip_merge_1.charge_unique_numbers.str.split('|')
explode_geocode = zip_merge_1.explode("charge_nums")

In [ ]:
len(explode_geocode)

In [ ]:
explode_cw = explode_geocode[["charge_nums","city_state","respondent_zip_nn", "state_county_fips", "county","respondent_county_nn"]]

In [ ]:
explode_cw.charge_nums = explode_cw.charge_nums.astype('str')

In [ ]:
explode_cw.to_csv(box_path+"geocoded_places.csv",
                 encoding = "utf-8-sig", index=False)